# LA Queue Report

## TPINT connection

In [20]:
#from /LA-tools/CapReports import CapTags as ct
import numpy as np
import datetime as dt
import pandas as pd
import os
import time
import math
import json
import cx_Oracle
base = os.getcwd()

In [24]:
''' connect to TPINT (or TPPE, LPSS) '''
''' parameters: {'query' : 'str of SQL query'} '''
''' output: {'result_list' : 'list of results returned from query'} '''

def OracleAPI(query):
    
    uid = 'tesi_interface'
    pwd = 'intint88'

    ip = '172.25.152.12'
    port = '1737'
    service_name = 'tpint.mytna.com'
    #dsn = cx_Oracle.makedsn(ip, port, service_name=service_name)
    
    print('querying...')
    
    con = cx_Oracle.connect(uid, pwd, service_name)
    cur = con.cursor()
    cur.execute(query)
    columns = [i[0] for i in cur.description]

    result_list = []
    
    for result in cur:
        result_list.append(result)
        i = len(result_list)
        if (i > 0 and i % 1000 == 0):
            print('done with {}.'.format(i))
    
    print('finished with {} results, outputting dataframe.'. format(len(result_list)))
    result = pd.DataFrame(result_list)
    result.columns = columns
    
    return(result)

In [25]:
queue = '''select 
 pr.quote_num "Pricing Request #",
pr.rev_num "Revision #",
PR.ACTIVE_FLG "Active",  
 pr.name "Pricing Request", 
  pr.created "PR Created", 
 pr.X_NO_OF_METERS_EST "Est. Meters",
acc.MARKET_CLASS_CD "Segmentation",
o.X_MAX_DEMAND "Max Demand (MW)",
acc.X_KEY_ACCOUNT_FLG "Key Account",
(select login from s_user@tpsbl usr, s_postn@tpsbl pos where pos.row_id = pr.SALES_REP_POSTN_ID and pos.pr_emp_id = usr.row_id) "PR Sales Rep", 
 pr.X_STATE "State",
pr.SUBTYPE_CD "PR Type",
pr.X_MARKET "Market",
stg.Name "Opportunity Sales Stage",
pr.OUT_DUE_DT "PR SLA Due Date",
acc.name "Legal Contracting Entity",
o.name "Opportunity",
pr.X_EST_MAX_DEMAND "Est. Demand (MWh)", 
  (select login from s_user@tpsbl usr where usr.row_id = pr.X_LOAD_ANALYST_ID ) "Load Analyst",
o.CHANNEL_TYPE_CD "Channel Type", 
 partner.alias_name "Channel Partner",
pr.X_PR_REQ_BY_DATE "PR Req By Date",
pr.EXCH_DT "Bid Due Date", 
 pr.HIST_AMT_DT "Bid Receipt Date",
(trunc(pr.EXCH_DT)-trunc(pr.HIST_AMT_DT))-2*FLOOR((trunc(pr.EXCH_DT)-trunc(pr.HIST_AMT_DT))/7)-DECODE(SIGN(TO_CHAR(trunc(pr.EXCH_DT),'D')-
        TO_CHAR(trunc(pr.HIST_AMT_DT),'D')),-1,2,0)+DECODE(TO_CHAR(trunc(pr.HIST_AMT_DT),'D'),7,1,0)-
        DECODE(TO_CHAR(trunc(pr.EXCH_DT),'D'),7,1,0)+1 as WorkDays,
pr.X_NON_EXECUTABLE_FLG "Non Executable Flag",
pr.VERIFY_DT "Loaded To PE On",
o.X_PROD_EXCEPTION_STATUS "Product Exception",
pr.X_NOTES "Notes",
pr.X_CONFIRM_TEXT "Load Analytics - Supply Notes",
qx.X_ATTRIB_53 "Off-Ramp Flag", 
 qx.X_ATTRIB_51 "EXP Flag",
decode( sign( trunc(pr.EXCH_DT) - decode(pr.active_flg, 'Y', NVL(trunc(act2.TODO_ACTL_END_DT), trunc(sysdate)), trunc(act2.TODO_ACTL_END_DT))) ,  -1, 'N', decode( sign( trunc(pr.EXCH_DT) - decode(pr.active_flg, 'Y', NVL(trunc(act2.TODO_ACTL_END_DT), trunc(sysdate)), trunc(act2.TODO_ACTL_END_DT))),'1', decode(act2.TODO_ACTL_END_DT, '', '', 'Y'), decode(act2.TODO_ACTL_END_DT, '', '', 'Y')))  "Met Bid Due Date",
  mml.utilitylist "PR Utilities",
trunc(pr.EXCH_DT) - trunc(pr.created) "Lead Time",
decode(act2.TODO_ACTL_END_DT, '', trunc(pr.EXCH_DT) -  trunc(sysdate), 0) "Days to Bid Due Date",
pr.rev_dt "Revision 1 Created",
act1.created "Gather Usage Created", act1.evt_stat_cd "Gather Usage Status",  act1.TODO_ACTL_END_DT "Gather Usage Completed" ,
act5.TODO_ACTL_END_DT "Load Analytic Created" , act5.evt_stat_cd "Load Analytic Status",   act5.TODO_ACTL_END_DT "Load Analytic Completed", 
act6.TODO_ACTL_END_DT "Sales Response Created" , act6.evt_stat_cd "Sales Response Status",   act6.TODO_ACTL_END_DT "Sales Response Completed", 
 pr.PRICING_DT "MML validated Date",
act3.TODO_ACTL_END_DT "Load Data Created" , act3.evt_stat_cd "Load Data Status",   act3.TODO_ACTL_END_DT "Load Data Completed",
act4.created "Escalation Started", act4.evt_stat_cd "Escalation Status", act4.TODO_ACTL_END_DT "Escalation Completed", 
 act2.created "Initial Pricing Started", act2.evt_stat_cd "Initial Pricing Status", act2.TODO_ACTL_END_DT "Initial Pricing Completed",  
 o.SUM_UPSIDE_AMT "ATCV",
metercount "Total Meter ID Count",
hu_requestcomplete_count + hu_datareceived_count "Count of HU Requested",
hu_datareceived_count "Count of HU Data Received",
hi_requestcomplete_count + hi_datareceived_count "Count of HI Requested",
hi_datareceived_count "Count of HI Data Received",
  decode(acc.X_KEY_ACCOUNT_FLG, 'Y', (select login from s_user@tpsbl usr, s_postn@tpsbl pos where pos.row_id = acc.pr_postn_id and pos.pr_emp_id = usr.row_id), '') "KAG Rep",
o.X_ANNUAL_USAGE "Annual Volume",
decode(decode( act2.created, '', 0, 10), 10, 10, decode(pr.PRICING_DT, '', 0, 10))+decode( act2.created, '', 0, 10) +   decode( act2.TODO_ACTL_END_DT, '', 0, 10) "Tracker Status"
from siebel.s_doc_quote@tpsbl pr, 
      siebel.s_evt_act@tpsbl act1, 
      siebel.s_evt_act@tpsbl act2, 
      siebel.s_evt_act@tpsbl act3,
      siebel.s_evt_act@tpsbl act4,
      siebel.s_evt_act@tpsbl act5,
      siebel.s_evt_act@tpsbl act6,
      siebel.s_opty@tpsbl o,
      siebel.s_org_ext@tpsbl acc,     
      siebel.s_doc_quote_x@tpsbl qx,
      siebel.s_stg@tpsbl stg,
      siebel.s_org_ext@tpsbl Partner,
      (select sbl_quote_id,
                      f_GetAggReportValue(sbl_quote_id,'UTILITYLIST') utilitylist,
                      f_GetAggReportValue(sbl_quote_id,'METERCOUNT') metercount,
                      f_GetAggReportValue(sbl_quote_id,'SCA_REQUEST_COMPLETE') hu_requestcomplete_count,
                      f_GetAggReportValue(sbl_quote_id,'SCA_DATA_RECEIVED') hu_datareceived_count,
                      f_GetAggReportValue(sbl_quote_id,'IDR_REQUEST_COMPLETE') hi_requestcomplete_count,
                      f_GetAggReportValue(sbl_quote_id,'IDR_DATA_RECEIVED') hi_datareceived_count
                      from alps_mml_pr_header) mml
where pr.row_id = act1.quote_id(+) and 
           act1.TODO_CD(+)= 'Gather Usage Data' and
           pr.row_id = act2.quote_id(+)  and 
           act2.TODO_CD (+) = 'Perform Initial Pricing' and 
           pr.row_id = act3.quote_id(+)  and 
           act3.TODO_CD (+) = 'Load Data and Perform VEE' and
           pr.row_id = act4.quote_id(+)  and 
           act4.TODO_CD (+) = 'Escalation Request'        
         and pr.row_id = act5.quote_id(+) 
         and act5.TODO_CD (+) = 'Load Analytic'       
         and pr.row_id = act6.quote_id(+)
         and act6.TODO_CD (+) = 'Pending Sales Response'      
           and O.CURR_STG_ID = stg.row_id
           and o.PR_PRTNR_ID = partner.row_id (+)
           and acc.row_id = O.PR_DEPT_OU_ID
           and o.row_id = pr.opty_id     
           and pr.row_id = QX.PAR_ROW_ID (+) 
           and pr.row_id = mml.sbl_quote_id(+)
           and nvl(qx.x_attrib_51, 'N') <> 'Y'         
           and trunc(pr.hist_amt_dt) >= (select case when extract(month from sysdate) between 1 and 9 then add_months(trunc(sysdate,'year'),0)
                                                                else add_months(trunc(sysdate,'year'),0)
                                                                 end from dual)'''

In [29]:
start = time.time()
print('updating queue...')
queue_query = OracleAPI(queue)
print('queue updated.')
end = time.time()
print('process took {} seconds.'.format(round(end - start)))

updating queue...
querying...
done with 1000.
done with 2000.
done with 3000.
done with 4000.
done with 5000.
done with 6000.
done with 7000.
done with 8000.
done with 9000.
done with 10000.
done with 11000.
done with 12000.
done with 13000.
finished with 13195 results, outputting dataframe.
queue updated.
process took 132 seconds.


In [30]:
queue_query.head()

,Pricing Request #,Revision #,Active,Pricing Request,PR Created,Est. Meters,Segmentation,Max Demand (MW),Key Account,PR Sales Rep,...,Initial Pricing Completed,ATCV,Total Meter ID Count,Count of HU Requested,Count of HU Data Received,Count of HI Requested,Count of HI Data Received,KAG Rep,Annual Volume,Tracker Status
0,1-IHDDT6,1.0,Y,DALTON SCHOOLS INC_CONED_3.19,2019-03-04 08:57:05,9,Bronze,0.626935,None,BRODRIGUEZ,...,2019-03-06 15:05:00,210000.0,9,9.0,9,1.0,1,None,2174,30
1,1-J7AIWR,1.0,Y,AEGON USA REALTY ADVISORS - JULY 2019,2019-07-15 13:54:12,1,Gold,1.119600,None,RBARRAGAN,...,2019-07-16 12:53:46,230000.0,1,1.0,1,1.0,1,None,4954,30
2,1-J3566H,1.0,Y,GARDEN STATE NUTRITIONAL_PSEG_12.2019 Renewal,2019-06-04 18:40:21,4,Silver,2.048961,None,JMAXSON,...,2019-06-10 11:05:40,747000.0,4,4.0,4,3.0,3,None,8685,30
3,1-IJ0XLQ,1.0,N,"RUDY'S FOOD PRODUCTS, INC._ONCOR_3.19_EIIO172",2019-03-13 09:48:25,4,Gold,3.625808,None,GFORD,...,2019-03-15 15:44:11,715000.0,4,4.0,4,3.0,3,None,18548,30
4,1-ILTJ5Q,1.0,Y,"RUDY'S FOOD PRODUCTS, INC._Oncor_3-2019_EILTJ56",2019-03-19 16:40:49,0,Gold,0.000000,None,GFORD,...,NaT,250000.0,3,3.0,3,3.0,3,None,0,20
